In [5]:
df = pd.read_csv('C:/Users/oyeye/Gas Market Simulation Model/ng.csv')

In [6]:
# Combined Impact of Government Incentives and Environmental Policies on Natural Gas Economics
def natural_gas_economics(df):
    def calc_natural_gas(row):
        Date = row['Date']
        Government_Incentives = row['Government_Incentives']
        Environmental_Policy_Strictness = row['Environmental_Policy_Strictness']
        gdp_growth_rate = row['GDP Growth Rate']
        Natural_Gas_Production_Mcf = row['Natural_Gas_Production_Mcf']
        Natural_Gas_Consumption_Mcf = row['Natural_Gas_Consumption_Mcf']
        base_incentive = 5000
        
        incentive_impact = (Government_Incentives - base_incentive) / base_incentive
        rate = 0.05
        threshold = np.random.uniform(rate, -rate)
        incentive_impact = min(incentive_impact, threshold)
        
        # Strictness Thresholds
        if Environmental_Policy_Strictness > 70:
            
            # -3% impact on production and consumption
            Natural_Gas_Production_Mcf = (1 - 0.03) * Natural_Gas_Production_Mcf
            Natural_Gas_Consumption_Mcf = (1 - 0.03) * Natural_Gas_Consumption_Mcf
            
        elif 50 < Environmental_Policy_Strictness < 70:
            Natural_Gas_Production_Mcf = (1 - 0.015) * Natural_Gas_Production_Mcf
            Natural_Gas_Consumption_Mcf = (1 - 0.015) * Natural_Gas_Consumption_Mcf
            
        elif Environmental_Policy_Strictness < 50:
            Natural_Gas_Production_Mcf = Natural_Gas_Production_Mcf
            Natural_Gas_Consumption_Mcf = Natural_Gas_Consumption_Mcf
            
        
        total_adjustment_factor = gdp_growth_rate + incentive_impact - Environmental_Policy_Strictness
        
        adjusted_production = Natural_Gas_Production_Mcf * (1 + total_adjustment_factor)
        adjusted_consumption = Natural_Gas_Consumption_Mcf * (1 + total_adjustment_factor)
        
        return pd.Series({'total_adjustment_factor' : total_adjustment_factor,
                         'adjusted_production' : adjusted_production,
                         'adjusted_consumption' : adjusted_consumption})
    
    result = df.apply(calc_natural_gas, axis = 1)
    df = pd.concat([df, result], axis = 1)
    return df

In [7]:
# Impact of Government Incentives and Environmental Policies on Natural Gas Prices
def natural_gas_price(df):
    df['govt_inc_change'] = df['Government_Incentives'].pct_change().fillna(0)
    
    def calc_gas_price(row):
        Date = row['Date']
        govt_inc_change = row['govt_inc_change']
        Environmental_Policy_Strictness = row['Environmental_Policy_Strictness']
        gdp_growth_rate = row['GDP Growth Rate']
        Average_Price_per_Mcf = row['Average_Price_per_Mcf']
        
        # reduce price by up to 3% due to increase in incentive
        if govt_inc_change > 0.1:
            incentive_adjustment = - 0.03
        
        # increase price by up to 3% due to decrease in incentive
        else:
            incentive_adjustment = 0.02
            
        # Environmental Regulation Impact
        if Environmental_Policy_Strictness > 75:
            env_reg_impact = 0.02
            
        elif 50 < Environmental_Policy_Strictness < 75:
            env_reg_impact = 0.01
            
        else:
            env_reg_impact = 0
            
        if gdp_growth_rate > 0:
            gdp_impact = -0.01
            
        else:
            gdp_impact = -0.01
            
        price_adjustment_factor = incentive_adjustment + gdp_impact + env_reg_impact
        
        adjusted_price = Average_Price_per_Mcf * (1 + price_adjustment_factor)
        
        return pd.Series({'base_price' : Average_Price_per_Mcf,
                         'adj_price' : adjusted_price})
    
    res1 = df.apply(calc_gas_price, axis = 1)
    df = pd.concat([df, res1], axis = 1)
    return df        

In [8]:
# Impact of Market Demand on Natural Gas Supply Sustainability
def natural_gas_supply_sustainability(df):
    def calc_supply(row):
        date = row['Date']
        Market_Demand_Factor = row['Market_Demand_Factor']
        Government_Incentives = row['Government_Incentives']
        Environmental_Policy_Strictness = row['Environmental_Policy_Strictness']
        Natural_Gas_Production_Mcf = row['Natural_Gas_Production_Mcf']
        
        # Demand Sensitivity
        if Market_Demand_Factor > 75:
            demand_adj = 0.1
            
        elif Market_Demand_Factor < 50:
            demand_adj = -0.1
            
        else:
            demand_adj = 0
            
        
        # Incentive Influence
        if Government_Incentives > 5000:
            inc_boost = -0.05
            
        elif Government_Incentives < 3000:
            inc_boost = 0.05
            
        else:
            inc_boost = 0
            
        # Environmental Regulation Impact
        if Environmental_Policy_Strictness > 75:
            policy_drag = 0.05
            
        elif 50 < Environmental_Policy_Strictness < 75:
            policy_drag = 0.025
            
        else:
            policy_drag = 0
            
        sustainability_index = Natural_Gas_Production_Mcf / (1 + demand_adj - inc_boost + policy_drag)
        
        return sustainability_index
    
    df['sustainability_index'] = df.apply(calc_supply, axis = 1)
    return df

In [9]:
# Impact of Market Dynamics on Natural Gas Imports and Exports
def natural_gas_scenarios(df):
    
    """
    Analyze the impact of government incentives, market demand, and economic conditions on the balance of natural gas imports and exports.

    Parameters:
    df (DataFrame): DataFrame containing natural gas trade data.

    Returns:
    DataFrame: Updated DataFrame with adjusted net export status.
    
    """
    
    def calc_scenarios(row):
        date = row['Date']
        Imports_Mcf = row['Imports_Mcf']
        Exports_Mcf = row['Exports_Mcf']
        Government_Incentives = row['Government_Incentives']
        Market_Demand_Factor = row['Market_Demand_Factor']
        gdp_growth_rate = row['GDP Growth Rate']
        
        # Incentive Adjustment for Exports
        if Government_Incentives > 5000:
            incentives_adj = 1.1
            
        else:
            incentives_adj = 0.95
        
        # Market Demand Influence on Imports
        if Market_Demand_Factor > 75:
            market_dd_inf = 0.9
            
        else:
            market_dd_inf = 1.1 
            
        # Economic Conditions Adjust Both Imports and Exports
        if gdp_growth_rate > 0:
            econ_boost = 1.05
            
        elif gdp_growth_rate < 0:
            econ_boost = 0.9
            
        else:
            econ_boost = 0
            
        adj_export_boost = Exports_Mcf * (incentives_adj + econ_boost)
        
        adj_import_boost = Imports_Mcf * (market_dd_inf + econ_boost)
        
        net_export_status = adj_export_boost - adj_import_boost
        
        return pd.Series({'adj_export_boost' : adj_export_boost,
                         'adj_import_boost' : adj_import_boost,
                         'net_export_status' : net_export_status})
    
    res4 = df.apply(calc_scenarios, axis = 1)
    df = pd.concat([df, res4], axis = 1)
    return df

In [14]:
df = pd.read_csv('C:/Users/oyeye/Gas Market Simulation Model/nat_gas.csv')

In [15]:
# Forecasting Natural Gas Prices with Government Incentives
def forecast_natural_gas_price(df):
    hist_avg_inc = df['Government_Incentives'].mean()
    n_periods = 12
    last_price = df['Average_Price_per_Mcf'].iloc[-1]
    forecast_prices = []
    
    for x in range(n_periods):
        current_incentive = df['Government_Incentives'].iloc[-1]
        incentive_change = (current_incentive - hist_avg_inc) / hist_avg_inc
        adj_factor = incentive_change * 0.05
        next_price = last_price * (1 - adj_factor)
        
        last_price = next_price
        forecast_prices.append(next_price)
        
    forecast_date = pd.date_range(start = df.index[-1] + pd.DateOffset(months = 1), periods = n_periods, freq = 'M')
    forecast_df = pd.DataFrame({'date' : forecast_date,
                               'future_prices' : forecast_prices})
    
    return forecast_df  

In [16]:
# Detailed Forecasting of Natural Gas Prices Based on Consumption Fluctuations
def ngprice_forecast(df):
    df['consumption_rate_change'] = df['Natural_Gas_Consumption_Mcf'].pct_change()
    df['policy_index_change'] = df['Environmental_Policy_Index'].pct_change()
    consumption_change = df['consumption_rate_change'].iloc[-1]
    latest_price = df['Average_Price_per_Mcf'].iloc[-1]
    gdp_growth_rate = df['GDP_Growth_Rate'].iloc[-1]
    policy_index_change = df['policy_index_change'].iloc[-1]
    base_price = df['Average_Price_per_Mcf'].iloc[-1]
    elasticity_factor = 0.02
    econ_imp_factor = 0.01
    env_imp_factor = 0.03
    n_periods = 12
    forecasted_price = []
    
    for x in range(n_periods):
        price_adj = latest_price * (consumption_change * elasticity_factor)
        econ_imp_adj = latest_price * (gdp_growth_rate * econ_imp_factor)
        env_imp_adj = latest_price * (policy_index_change * env_imp_factor)
        
        updated_price = latest_price + price_adj + econ_imp_adj - env_imp_adj
        
        latest_price = updated_price
        
        forecasted_price.append(updated_price)

        
    
    date = pd.date_range(start = df.index[-1] + pd.DateOffset(months = 1), periods = n_periods, freq = 'M')
    forecasted_df = pd.DataFrame({'date' : date,
                                 'price' : forecasted_price})
    
    return forecasted_df